```{contents}
```
## **Live State Inspection in LangGraph**

**Live state inspection** is the ability to **observe, pause, modify, and reason about the internal state of a running LangGraph execution** in real time.
It is a foundational feature for **debugging, human oversight, safety, and production operations** in complex LLM workflows.

---

### **1. Why Live State Inspection Is Needed**

LangGraph systems are:

* **Stateful**
* **Multi-step**
* **Often cyclic**
* **Possibly autonomous**

Without visibility into the evolving state, these systems become **opaque and unsafe**.

| Capability                 | Why It Matters         |
| -------------------------- | ---------------------- |
| Observe intermediate state | Debug reasoning        |
| Pause execution            | Prevent unsafe actions |
| Modify state               | Correct errors         |
| Audit decisions            | Compliance             |
| Resume from checkpoint     | Fault recovery         |

---

### **2. Conceptual Model**

```
Execution Engine
      ↓
 [ Node A ] → State₁
      ↓
 [ Node B ] → State₂   ←── Inspect Here
      ↓
 [ Node C ] → State₃
```

The **state object** is the single source of truth and can be inspected at **every step**.

---

### **3. State as the Inspection Surface**

All inspection occurs on the **shared typed state**:

```python
class State(TypedDict):
    messages: list
    plan: str
    risk: float
    approved: bool
```

Every node emits a **partial update** to this state, making changes easy to observe.

---

### **4. Enabling Live Inspection with Checkpointing**

LangGraph provides **checkpointing** to expose execution progress.

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("state.db")

graph = builder.compile(checkpointer=checkpointer)
```

Each step is now stored and can be inspected at any time.

---

### **5. Streaming Execution with State Visibility**

```python
for event in graph.stream({"input": "Review contract"}):
    print(event)
```

Typical output:

```
{'node': 'analyze', 'state': {...}}
{'node': 'risk_check', 'state': {...}}
{'node': 'approval', 'state': {...}}
```

This provides **live execution telemetry**.

---

### **6. Pausing & Human Inspection**

Define an **interrupt node**:

```python
from langgraph.graph import INTERRUPT

builder.add_edge("risk_check", INTERRUPT)
```

Execution halts and exposes state:

```python
snapshot = graph.get_state(thread_id)
print(snapshot)
```

Human can modify:

```python
graph.update_state(thread_id, {"approved": True})
graph.resume(thread_id)
```

---

### **7. Production Workflow Example**

**Compliance Review System**

```
Ingest → Analyze → RiskScore → [Interrupt] → HumanApprove → Finalize
```

At the interrupt:

* Inspector sees current **state**
* Adjusts decision fields
* Allows safe continuation

---

### **8. Debugging Use Case**

| Task                  | Live Inspection Role           |
| --------------------- | ------------------------------ |
| Trace wrong output    | Examine intermediate reasoning |
| Detect hallucination  | Inspect evidence fields        |
| Fix tool misuse       | Adjust tool input              |
| Prevent infinite loop | Modify control flags           |

---

### **9. Safety & Governance Benefits**

| Benefit        | Description               |
| -------------- | ------------------------- |
| Explainability | See how answers form      |
| Control        | Override unsafe decisions |
| Auditability   | Full state history        |
| Compliance     | Human verification        |
| Recovery       | Resume from failure       |

---

### **10. Mental Model**

Live state inspection turns LangGraph into a:

> **Transparent, controllable execution machine**

rather than a black-box chain.


### Demonstration

In [9]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver

# ----------------------------
# 1. Define State
# ----------------------------

class State(TypedDict):
    text: str
    risk: int
    approved: bool

# ----------------------------
# 2. Define Nodes
# ----------------------------

def analyze(state: State):
    print("Analyzing...")
    return {"risk": len(state["text"]) % 10}

def risk_check(state: State):
    print("Risk score:", state["risk"])
    return {}

def finalize(state: State):
    print("Finalizing...")
    return {"text": state["text"] + " [FINALIZED]"}

# ----------------------------
# 3. Build Graph
# ----------------------------

builder = StateGraph(State)

builder.add_node("analyze", analyze)
builder.add_node("risk_check", risk_check)
builder.add_node("finalize", finalize)

builder.set_entry_point("analyze")
builder.add_edge("analyze", "risk_check")

builder.add_conditional_edges(
    "risk_check",
    lambda s: "finalize" if s["approved"] else END,
    {"finalize": "finalize", END: END}
)

# ----------------------------
# 4. Compile with Checkpoint + Interrupt
# ----------------------------

checkpointer = InMemorySaver()

graph = builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["finalize"]
)

# ----------------------------
# 5. First Run
# ----------------------------

config = {"configurable": {"thread_id": "demo"}}

for event in graph.stream({"text": "Review this contract", "approved": False}, config):
    print("EVENT:", event)

# ----------------------------
# 6. Live State Inspection
# ----------------------------

print("\n--- LIVE STATE ---")
print(graph.get_state(config))

# ----------------------------
# 7. Human Intervention
# ----------------------------

print("\nApproving manually...\n")
graph.update_state(config, {"approved": True})

# ----------------------------
# 8. Continue Execution
# ----------------------------

for event in graph.stream(None, config):
    print("EVENT:", event)

# ----------------------------
# 9. Final State
# ----------------------------

print("\n--- FINAL STATE ---")
print(graph.get_state(config))


Analyzing...
EVENT: {'analyze': {'risk': 0}}
Risk score: 0
EVENT: {'risk_check': None}

--- LIVE STATE ---
StateSnapshot(values={'text': 'Review this contract', 'risk': 0, 'approved': False}, next=(), config={'configurable': {'thread_id': 'demo', 'checkpoint_ns': '', 'checkpoint_id': '1f0e4c51-3133-6d59-8002-9eaa6ddcce62'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-29T14:45:52.820360+00:00', parent_config={'configurable': {'thread_id': 'demo', 'checkpoint_ns': '', 'checkpoint_id': '1f0e4c51-312f-6d93-8001-4efe71a5d0d5'}}, tasks=(), interrupts=())

Approving manually...

Finalizing...
EVENT: {'finalize': {'text': 'Review this contract [FINALIZED]'}}

--- FINAL STATE ---
StateSnapshot(values={'text': 'Review this contract [FINALIZED]', 'risk': 0, 'approved': True}, next=(), config={'configurable': {'thread_id': 'demo', 'checkpoint_ns': '', 'checkpoint_id': '1f0e4c51-313e-68c6-8004-55510ddaeaf8'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, cr